<a href="https://colab.research.google.com/github/R3gm/Colab-resources/blob/main/ImageColorizer_DISCO_Disentangled_Image_Colorization_via_Global_Anchors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Original repository:
https://github.com/MenghanXia/DisentangledColorization

Config spaces in colab for `machinelearnear` original colab in 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/machinelearnear/open-hf-spaces-in-studiolab/blob/main/run_google_colab.ipynb)

In [1]:
#@title DISCO configuration and run in gradio in one clic 🚀 Running on public URL
# source: https://www.youtube.com/c/machinelearnear

import os
import sys
import shutil
import subprocess
import re

try:
    import gradio as gr
except ImportError:
    subprocess.run(["pip", "install", "gradio"])
    import gradio as gr

from os.path import exists as path_exists
from pathlib import Path
from typing import Dict

newline, bold, unbold = "\n", "\033[1m", "\033[0m"

class RepoHandler:
    def __init__(self, repo_url: str) -> None:
        """
        Initialize the RepoHandler class with the provided repository URL and requirements file.
        Args:
        - repo_url: URL of the git repository to clone.
        Returns:
        None
        """
        self.is_google_colab = False
        self.fix_for_gr, self.fix_for_st = None, None
        self.app_file = "app.py"
        if 'google.colab' in str(get_ipython()):
            print(f'{bold}Running on: "Google Colab"{unbold}')
            self.is_google_colab = True
        else:
            print(f'{bold}Running on: Local or "SM Studio Lab"{unbold}')
        self.repo_url = repo_url
        self.repo_name = self.repo_url.split('/')[-1]

    def __str__(self):
        if os.path.exists(self.repo_name):
            return self.retrieve_readme(f'{self.repo_name}/README.md')
        else:
            print(f"{bold}The repo '{self.repo_name}' has not been cloned yet.{unbold}")
            return None
            
    def retrieve_readme(self, filename) -> Dict:
        readme = {}
        if path_exists(filename):
            with open(filename) as f:
                for line in f:
                    if line.find('http') > 0: continue
                    if not line.find(':') > 0 or 'Check' in line: continue
                    (k,v) = line.split(':')
                    readme[(k)] = v.strip().replace('\n','')
        else:
            print(f"{bold}No 'readme.md' file{unbold}")
            
        return readme
        
    def clone_repo(self, overwrite=False) -> None:
        """
        Clone the git repository specified in the repo_url attribute.
        Returns:
        None
        """
        # Check if repository has already been cloned locally
        if overwrite and os.path.exists(self.repo_name): 
            try:
                shutil.rmtree(self.repo_name)
            except OSError as e:
                print("Error: %s - %s." % (e.filename, e.strerror))
        if os.path.exists(self.repo_name):
            print(f"{bold}Repository '{self.repo_name}' has already been cloned.{unbold}")
        else:
            print(f"{bold}Cloning repo... may take a few minutes... remember to set your Space to 'public'...{unbold}")
            subprocess.run(["apt-get", "install", "git-lfs"])
            subprocess.run(["git", "lfs", "install", "--system", "--skip-repo"])
            subprocess.run(["git", "clone", "--recurse-submodules", self.repo_url])

    def install_requirements(self, requirements_file: str = None, install_xformers: bool = False) -> None:
        """
        Install the requirements specified in the requirements_file attribute.
        
        Args:
        - requirements_file: Name of the file containing the requirements to install. This file must be 
        located in the root directory of the repository. Defaults to "requirements.txt".
        Returns:
        None
        """
        if not requirements_file: requirements_file = f"{self.repo_name}/requirements.txt"
        
        # install requirements
        print(f"{bold}Installing requirements... may take a few minutes...{unbold}")
        subprocess.run(["pip", "install", "-r", requirements_file])
        if install_xformers: self.install_xformers()

    def run_web_demo(self, aws_domain=None, aws_region=None) -> None:
        """
        Launch the Gradio or Streamlit web demo for the cloned repository.
        Works with Google Colab or SageMaker Studio Lab.
        Returns:
        None
        """
        import torch
        if torch.cuda.is_available(): print(f"{bold}Using: {unbold}{self.get_gpu_memory_map()}")
        else: print(f"{bold}Not using the GPU{unbold}")
        
        readme = self.__str__()
        self.app_file = readme["app_file"]
        print(f"{bold}Demo: `{readme['title']}`{newline}{unbold}")
        print(f"{bold}Downloading models... might take up to 10 minutes to finish...{unbold}")
        print(f"{bold}Once finished, click the link below to open your application (in SM Studio Lab):{newline}{unbold}")
        if all([aws_domain, aws_region]):
              print(f'{bold}https://{aws_domain}.studio.{aws_region}.sagemaker.aws/studiolab/default/jupyter/proxy/6006/{unbold}')
                
        self.unset_environment_variables()
        
        if readme["sdk"] == 'gradio':
            gr.close_all()
            if not self.is_google_colab:
                !export GRADIO_SERVER_PORT=6006 && cd $self.repo_name && python $self.app_file
                # os.system(f'export GRADIO_SERVER_PORT=6006 && cd {self.repo_name} && python {readme["app_file"]}')
            else:
                new_filename = self.replace_gradio_launcher(f'{self.repo_name}/{readme["app_file"]}')
                !cd $self.repo_name && python $new_filename
                # os.system(f'cd {self.repo_name} && python {readme["app_file"]}')
        elif readme["title"] == 'streamlit':
            if not self.is_google_colab:
                !cd $self.repo_name && streamlit run $self.app_file --server.port 6006
                # os.system(f'cd {self.repo_name} && streamlit run {readme["app_file"]} --server.port 6006')
            else:
                !cd $self.repo_name && streamlit run $self.app_file
                # os.system(f'cd {self.repo_name} && streamlit run {readme["app_file"]}')
        else:
            print('This notebook will not work with static apps hosted on "Spaces"')

    def get_gpu_memory_map(self) -> Dict[str, int]:
        """Get the current gpu usage.
        Return:
            A dictionary in which the keys are device ids as integers and
            values are memory usage as integers in MB.
        """
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=name,memory.total,memory.free", "--format=csv,noheader",],
            encoding="utf-8",
            # capture_output=True,          # valid for python version >=3.7
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,  # for backward compatibility with python version 3.6
            check=True,
        )
        # Convert lines into a dictionary, return f"{}"
        gpu_memory = [x for x in result.stdout.strip().split(os.linesep)]
        gpu_memory_map = {f"gpu_{index}": memory for index, memory in enumerate(gpu_memory)}
        
        return gpu_memory_map

    def replace_gradio_launcher(self, old_filename) -> str:
        # Read the contents of the file
        with open(old_filename, "r") as f:
            contents = f.read()
        # Define the regular expression pattern
        pattern = r"\.launch\((.*?)\)"
        # Use the sub method to replace the text
        contents = re.sub(pattern, ".launch(share=True)", contents)
        # Write the modified contents back to the file
        new_filename = Path(old_filename).parent / f"{Path(old_filename).stem}_modified.py"
        with open(new_filename, "w") as f:
            f.write(contents)

        return new_filename.name
    
    def unset_environment_variables(self) -> None:
        os.unsetenv("SHARED_UI")
        os.environ.pop("SHARED_UI", None)
        
        os.unsetenv("IS_SHARED")
        os.environ.pop("IS_SHARED", None)

    def install_xformers(self) -> None:
        from subprocess import getoutput
        from IPython.display import HTML
        from IPython.display import clear_output
        import time

        subprocess.run(["pip", "install", "-U", "--pre", "triton"])

        s = getoutput('nvidia-smi')
        if 'T4' in s: gpu = 'T4'
        elif 'P100' in s: gpu = 'P100'
        elif 'V100' in s: gpu = 'V100'
        elif 'A100' in s: gpu = 'A100'

        while True:
            try: 
                gpu=='T4'or gpu=='P100'or gpu=='V100'or gpu=='A100'
                break
            except:
                pass
            print(f'{bold} Seems that your GPU is not supported at the moment.{unbold}')
            time.sleep(5)

        if (gpu=='T4'): 
            precompiled_wheels = "https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/T4/xformers-0.0.13.dev0-py3-none-any.whl"
        elif (gpu=='P100'): 
            precompiled_wheels = "https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/P100/xformers-0.0.13.dev0-py3-none-any.whl"
        elif (gpu=='V100'): 
            precompiled_wheels = "https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/V100/xformers-0.0.13.dev0-py3-none-any.whl"
        elif (gpu=='A100'): 
            precompiled_wheels = "https://github.com/TheLastBen/fast-stable-diffusion/raw/main/precompiled/A100/xformers-0.0.13.dev0-py3-none-any.whl"

        subprocess.run(["pip", "install", "-q", precompiled_wheels])

app = RepoHandler(
    repo_url='https://huggingface.co/spaces/menghanxia/disco'
)
app.clone_repo(overwrite=True)
app.install_requirements()
app.run_web_demo()

Running on: "Google Colab"
Cloning repo... may take a few minutes... remember to set your Space to 'public'...
Installing requirements... may take a few minutes...
Using: {'gpu_0': 'Tesla T4, 15360 MiB, 15098 MiB'}
Demo: `Disco`

Once finished, click the link below to open your application (in SM Studio Lab):

--2023-05-07 22:01:59--  https://huggingface.co/menghanxia/disco/resolve/main/disco-beta.pth.rar
Resolving huggingface.co (huggingface.co)... 18.160.249.70, 18.160.249.31, 18.160.249.78, ...
Connecting to huggingface.co (huggingface.co)|18.160.249.70|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/e9/e6/e9e63bf8a01864a93436a8bd5ae17cd37d40253953af0912d8399e1d56e6403d/b9d29e79c246813d4b07aa53e25d14a9f0d58f5a591a8f80a231d45514525db6?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27disco-beta.pth.rar%3B+filename%3D%22disco-beta.pth.rar%22%3B&response-content-type=application%2Fvnd.rar&Expires=168375

In [2]:
!pip freeze | sed 's/==/==/g' > log_versions.txt

In [ ]:
#@title log versions
%%writefile log_versions.txt
absl-py==1.4.0
addict==2.4.0
aiofiles==23.1.0
aiohttp==3.8.4
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.6.2
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
async-timeout==4.0.2
attrs==23.1.0
audioread==3.0.0
autograd==1.5
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
CacheControl==0.12.11
cached-property==1.5.2
cachetools==5.3.0
catalogue==2.0.8
certifi==2022.12.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.5
contextlib2==0.6.0.post1
contourpy==1.0.7
convertdate==2.4.0
cryptography==40.0.2
cufflinks==0.17.3
cupy-cuda11x==11.0.0
cvxopt==1.3.0
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.34
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-python==1.2.16
debugpy==1.6.6
decorator==4.4.2
defusedxml==0.7.1
distributed==2022.12.1
dlib==19.24.1
dm-tree==0.1.8
docutils==0.16
dopamine-rl==4.0.6
duckdb==0.7.1
earthengine-api==0.1.350
easydict==1.10
ecos==2.0.12
editdistance==0.6.2
en-core-web-sm @ https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.5.0/en_core_web_sm-3.5.0-py3-none-any.whl
entrypoints==0.4
ephem==4.1.4
et-xmlfile==1.1.0
etils==1.2.0
etuples==0.3.8
exceptiongroup==1.1.1
fastai==2.7.12
fastapi==0.95.1
fastcore==1.5.29
fastdownload==0.0.7
fastjsonschema==2.16.3
fastprogress==1.0.3
fastrlock==0.8.1
ffmpy==0.3.0
filelock==3.12.0
firebase-admin==5.3.0
Flask==2.2.4
flatbuffers==23.3.3
flax==0.6.9
folium==0.14.0
fonttools==4.39.3
frozendict==2.3.7
frozenlist==1.3.3
fsspec==2023.4.0
future==0.18.3
gast==0.4.0
GDAL==3.3.2
gdown==4.6.6
gensim==4.3.1
geographiclib==2.0
geopy==2.3.0
gin-config==0.5.0
glob2==0.7
google==2.0.3
google-api-core==2.11.0
google-api-python-client==2.84.0
google-auth==2.17.3
google-auth-httplib2==0.1.0
google-auth-oauthlib==1.0.0
google-cloud-bigquery==3.9.0
google-cloud-bigquery-storage==2.19.1
google-cloud-core==2.3.2
google-cloud-datastore==2.15.1
google-cloud-firestore==2.11.0
google-cloud-language==2.9.1
google-cloud-storage==2.8.0
google-cloud-translate==3.11.1
google-colab @ file:///colabtools/dist/google-colab-1.0.0.tar.gz
google-crc32c==1.5.0
google-pasta==0.2.0
google-resumable-media==2.5.0
googleapis-common-protos==1.59.0
googledrivedownloader==0.4
gradio==3.28.3
gradio_client==0.2.0
graphviz==0.20.1
greenlet==2.0.2
grpcio==1.54.0
grpcio-status==1.48.2
gspread==3.4.2
gspread-dataframe==3.0.8
gym==0.25.2
gym-notices==0.0.8
h11==0.14.0
h5netcdf==1.1.0
h5py==3.8.0
hijri-converter==2.3.1
holidays==0.23
holoviews==1.15.4
html5lib==1.1
httpcore==0.17.0
httpimport==1.3.0
httplib2==0.21.0
httpx==0.24.0
huggingface-hub==0.14.1
humanize==4.6.0
hyperopt==0.2.7
idna==3.4
imageio==2.25.1
imageio-ffmpeg==0.4.8
imagesize==1.4.1
imbalanced-learn==0.10.1
imgaug==0.4.0
importlib-resources==5.12.0
imutils==0.5.4
inflect==6.0.4
iniconfig==2.0.0
intel-openmp==2023.1.0
ipykernel==5.5.6
ipython==7.34.0
ipython-genutils==0.2.0
ipython-sql==0.4.1
ipywidgets==7.7.1
itsdangerous==2.1.2
jax==0.4.8
jaxlib @ https://storage.googleapis.com/jax-releases/cuda11/jaxlib-0.4.7+cuda11.cudnn86-cp310-cp310-manylinux2014_x86_64.whl
jieba==0.42.1
Jinja2==3.1.2
joblib==1.2.0
jsonpickle==3.0.1
jsonschema==4.3.3
jupyter-client==6.1.12
jupyter-console==6.1.0
jupyter-server==1.24.0
jupyter_core==5.3.0
jupyterlab-pygments==0.2.2
jupyterlab-widgets==3.0.7
kaggle==1.5.13
keras==2.12.0
kiwisolver==1.4.4
korean-lunar-calendar==0.3.1
langcodes==3.3.0
lazy_loader==0.2
libclang==16.0.0
librosa==0.10.0.post2
lightgbm==3.3.5
linkify-it-py==2.0.2
lit==16.0.2
llvmlite==0.39.1
locket==1.0.0
logical-unification==0.4.5
lpips==0.1.4
LunarCalendar==0.0.9
lxml==4.9.2
Markdown==3.4.3
markdown-it-py==2.2.0
MarkupSafe==2.1.2
matplotlib==3.7.1
matplotlib-inline==0.1.6
matplotlib-venn==0.11.9
mdit-py-plugins==0.3.3
mdurl==0.1.2
miniKanren==1.0.3
missingno==0.5.2
mistune==0.8.4
mizani==0.8.1
mkl==2019.0
ml-dtypes==0.1.0
mlxtend==0.14.0
more-itertools==9.1.0
moviepy==1.0.3
mpmath==1.3.0
msgpack==1.0.5
multidict==6.0.4
multipledispatch==0.6.0
multitasking==0.0.11
murmurhash==1.0.9
music21==8.1.0
natsort==8.3.1
nbclient==0.7.4
nbconvert==6.5.4
nbformat==5.8.0
nest-asyncio==1.5.6
networkx==3.1
nibabel==3.0.2
nltk==3.8.1
notebook==6.4.8
numba==0.56.4
numexpr==2.8.4
numpy==1.22.4
oauth2client==4.1.3
oauthlib==3.2.2
opencv-contrib-python==4.7.0.72
opencv-python==4.7.0.72
opencv-python-headless==4.7.0.72
openpyxl==3.0.10
opt-einsum==3.3.0
optax==0.1.5
orbax-checkpoint==0.2.1
orjson==3.8.12
osqp==0.6.2.post8
packaging==23.1
palettable==3.3.3
pandas==1.5.3
pandas-datareader==0.10.0
pandas-gbq==0.17.9
pandocfilters==1.5.0
panel==0.14.4
param==1.13.0
parso==0.8.3
partd==1.4.0
pathlib==1.0.1
pathy==0.10.1
patsy==0.5.3
pep517==0.13.0
pexpect==4.8.0
pickleshare==0.7.5
Pillow==8.4.0
pip-tools==6.6.2
platformdirs==3.3.0
plotly==5.13.1
plotnine==0.10.1
pluggy==1.0.0
polars==0.17.3
pooch==1.6.0
portpicker==1.3.9
prefetch-generator==1.0.3
preshed==3.0.8
prettytable==0.7.2
proglog==0.1.10
progressbar2==4.2.0
prometheus-client==0.16.0
promise==2.3
prompt-toolkit==3.0.38
prophet==1.1.2
proto-plus==1.22.2
protobuf==3.20.3
psutil==5.9.5
psycopg2==2.9.6
ptyprocess==0.7.0
py-cpuinfo==9.0.0
py4j==0.10.9.7
pyarrow==9.0.0
pyasn1==0.5.0
pyasn1-modules==0.3.0
pycocotools==2.0.6
pycparser==2.21
pyct==0.5.0
pydantic==1.10.7
pydata-google-auth==1.7.0
pydot==1.4.2
pydot-ng==2.0.0
pydotplus==2.0.2
PyDrive==1.3.1
pydub==0.25.1
pyerfa==2.0.0.3
pygame==2.3.0
Pygments==2.14.0
PyGObject==3.36.0
pymc==5.1.2
PyMeeus==0.5.12
pymystem3==0.2.0
PyOpenGL==3.1.6
pyparsing==3.0.9
pyrsistent==0.19.3
PySocks==1.7.1
pytensor==2.10.1
pytest==7.2.2
python-apt==0.0.0
python-dateutil==2.8.2
python-louvain==0.16
python-multipart==0.0.6
python-slugify==8.0.1
python-utils==3.5.2
pytz==2022.7.1
pytz-deprecation-shim==0.1.0.post0
pyviz-comms==2.2.1
PyWavelets==1.4.1
PyYAML==6.0
pyzmq==23.2.1
qdldl==0.1.7
qudida==0.0.4
regex==2022.10.31
requests==2.27.1
requests-oauthlib==1.3.1
requests-unixsocket==0.2.0
rich==13.3.4
rpy2==3.5.5
rsa==4.9
scikit-image==0.19.3
scikit-learn==1.2.2
scipy==1.10.1
scs==3.2.3
seaborn==0.12.2
semantic-version==2.10.0
Send2Trash==1.8.0
shapely==2.0.1
six==1.16.0
sklearn-pandas==2.2.0
smart-open==6.3.0
sniffio==1.3.0
snowballstemmer==2.2.0
sortedcontainers==2.4.0
soundfile==0.12.1
soupsieve==2.4.1
soxr==0.3.5
spacy==3.5.2
spacy-legacy==3.0.12
spacy-loggers==1.0.4
Sphinx==3.5.4
sphinxcontrib-applehelp==1.0.4
sphinxcontrib-devhelp==1.0.2
sphinxcontrib-htmlhelp==2.0.1
sphinxcontrib-jsmath==1.0.1
sphinxcontrib-qthelp==1.0.3
sphinxcontrib-serializinghtml==1.1.5
SQLAlchemy==2.0.10
sqlparse==0.4.4
srsly==2.4.6
starlette==0.26.1
statsmodels==0.13.5
sympy==1.11.1
tables==3.8.0
tabulate==0.8.10
tblib==1.7.0
tenacity==8.2.2
tensorboard==2.12.2
tensorboard-data-server==0.7.0
tensorboard-plugin-wit==1.8.1
tensorboardX==2.6
tensorflow==2.12.0
tensorflow-datasets==4.8.3
tensorflow-estimator==2.12.0
tensorflow-gcs-config==2.12.0
tensorflow-hub==0.13.0
tensorflow-io-gcs-filesystem==0.32.0
tensorflow-metadata==1.13.1
tensorflow-probability==0.19.0
tensorstore==0.1.36
termcolor==2.3.0
terminado==0.17.1
text-unidecode==1.3
textblob==0.17.1
tf-slim==1.1.0
thinc==8.1.9
threadpoolctl==3.1.0
tifffile==2023.4.12
tinycss2==1.2.1
toml==0.10.2
tomli==2.0.1
toolz==0.12.0
torch @ https://download.pytorch.org/whl/cu118/torch-2.0.0%2Bcu118-cp310-cp310-linux_x86_64.whl
torchaudio @ https://download.pytorch.org/whl/cu118/torchaudio-2.0.1%2Bcu118-cp310-cp310-linux_x86_64.whl
torchdata==0.6.0
torchsummary==1.5.1
torchtext==0.15.1
torchvision @ https://download.pytorch.org/whl/cu118/torchvision-0.15.1%2Bcu118-cp310-cp310-linux_x86_64.whl
tornado==6.2
tqdm==4.65.0
traitlets==5.7.1
triton==2.0.0
tweepy==4.13.0
typer==0.7.0
typing_extensions==4.5.0
tzdata==2023.3
tzlocal==4.3
uc-micro-py==1.0.2
uritemplate==4.1.1
urllib3==1.26.15
uvicorn==0.22.0
vega-datasets==0.9.0
wasabi==1.1.1
wcwidth==0.2.6
webcolors==1.13
webencodings==0.5.1
websocket-client==1.5.1
websockets==11.0.3
Werkzeug==2.3.0
widgetsnbextension==3.6.4
wordcloud==1.8.2.2
wrapt==1.14.1
xarray==2022.12.0
xarray-einstats==0.5.1
xgboost==1.7.5
xlrd==2.0.1
yapf==0.33.0
yarl==1.9.2
yellowbrick==1.5
yfinance==0.2.18
zict==3.0.0
zipp==3.15.0
